<a href="https://colab.research.google.com/github/hzhang0311/Wordcloud/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import dataset: use jasa1
data(heart, package="survival")


In [ ]:
heart1 <- heart[-which(heart$event==0 & heart$transplant==0),]
heart1$u = ifelse(heart1$transplant == 0, heart1$stop+1, heart1$start)
head(heart1)

,start,stop,event,age,year,surgery,transplant,id,u
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
1,0,50,1,-17.155373,0.1232033,0,0,1,51
2,0,6,1,3.835729,0.2546201,0,0,2,7
4,1,16,1,6.297057,0.2655715,0,1,3,1
6,36,39,1,-7.737166,0.4900753,0,1,4,36
7,0,18,1,-27.214237,0.6078029,0,0,5,19
8,0,3,1,6.595483,0.7008898,0,0,6,4


In [ ]:
u <- heart1$u
t <- heart1$stop
n <- length(t)
sigma <- heart1$event

In [ ]:
heart1[25,]

,start,stop,event,age,year,surgery,transplant,id,u
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
40,25,1800,0,-14.77618,1.574264,0,1,25,25


In [ ]:
summary(heart1$stop)
length(which(heart1$event==0))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    1.0    35.5    90.0   303.9   383.5  1800.0 

[1] 24

In [ ]:
#t-u vector #Xi(ti) vector
compare_ti_ui <- (t-u)
sum(compare_ti_ui <=0)
length(compare_ti_ui)
Xi_ti <- compare_ti_ui
Xi_ti[Xi_ti <= 0] <- 0
Xi_ti[Xi_ti >0 ] <- 1

[1] 30

[1] 99

In [ ]:
#compare ti_uj matrix
#Xj_ti matrix
compare_ti_uj <- outer(t,u,FUN=function(x,y)(x-y)) #n*n matrix
Xj_ti <- compare_ti_uj
Xj_ti[Xj_ti <= 0] <- 0
Xj_ti[Xj_ti >0 ] <- 1

In [ ]:
#Yj_ti matrix
#Yj_ti <- outer(t, t, FUN=function(x,y)(x-y))
#Yj_ti[Yj_ti <= 0] <- 0
#Yj_ti[Yj_ti > 0] <- 1

In [ ]:
#reconstruct Yj_ti matrix
Yj_ti <- outer(t, t, FUN=function(x,y)(x-y))
Yj_ti[Yj_ti < 0] <- 0
Yj_ti[Yj_ti > 0] <- 1
diag(Yj_ti) <- ifelse(heart1$event== 0, 1, 0)

In [ ]:
#Zj_ti matrix #seems like this needs to be constructed within each function
#Zj_ti <- Xj_ti*exp(-gamma * (compare_ti_uj))

In [ ]:
#logLikelihood
logL <- function(params, sigma){
  #n <- length(t) defined before the three functions
  b <- params[1]
  r <- params[2]
  
  Zj_ti <- Xj_ti*exp(-r * (compare_ti_uj))

  #first part of log Likelihood
  
  first_part <- sum( sigma * b * Xi_ti * exp(-r*compare_ti_ui) )
  print(first_part)
  
  #second part of log Likelihood
  inner_part <- colSums(Yj_ti * exp(b*Zj_ti)) 
  print(inner_part)
  log_part <- log(inner_part)
  print(log_part)

  second_part <- sum(sigma*log_part)
  print(second_part)
  
  return ((-first_part + second_part))
}

In [ ]:
logL(c(-0.294,0.001),sigma)

[1] -11.06173
 [1] 52.8786738 70.3454989 65.9515487 56.5222159 64.2044141 73.3745693
 [7] 13.5675382 54.9723238 41.1111826 50.7823512 34.6982347 69.8240060
[13] 42.4042085  3.7570317 78.6438473 26.0336866 59.0017074 54.4624470
[19] 57.9893233 61.9790327  6.5261736 52.2916744 12.7633580 29.7563979
[25]  0.9513929 28.0805394 44.9659926 59.7595777 10.9883974 65.9777885
[31] 44.6803250  1.8888250  2.8312567 68.0182454 39.1561168 49.2023678
[37] 71.8755916 51.6393325  3.7560888  5.5974542 73.3745693 76.3909712
[43] 54.9723238 53.9177980  7.4292673 45.4041523 68.8088474  6.5132156
[49]  8.2718256 26.8190755 37.9995011 31.5150336 73.3745693 50.0460974
[55] 10.0562435 34.6978439 22.8293743 10.9268804 47.8372044 76.3909712
[61] 46.4663525 12.7348228 16.9994418 43.9626218 60.2889492 26.7218650
[67] 47.8372044 15.3408397 61.4001747 16.1570828 17.0532890 40.8101215
[73] 65.1467928 76.3909712 18.6536360 62.6724018 19.0719444 39.7590610
[79] 20.3621932 21.2383787 43.0434283 25.1968878 72.3610608 22.

[1] 289.4474

In [ ]:
g_gamma = function(params, sigma, t, u){ 
  n <- length(t)
  b <- params[1]
  r <- params[2]

  Zj_ti <- Xj_ti*exp(-r * (compare_ti_uj))
  first_part <- (-1) * sum(sigma * b * diag(Xj_ti) * compare_ti_uj * exp(-r*compare_ti_uj) )
  print(first_part)
  
  #num <- colSums(Yj_ti * Xj_ti * exp(b * Zj_ti- r * compare_ti_uj))
  num <- colSums(Yj_ti * Xj_ti * exp(b * Zj_ti - r * compare_ti_uj))
  denom <- colSums(Yj_ti * exp(b * Zj_ti))
  sec_part <- b * sum(sigma * (num / denom))
  print(sec_part)
  
  return(first_part + sec_part)
}

In [ ]:
g_gamma(c(0,0.001),sigma, t,u)

[1] 0
[1] 0


[1] 0

In [ ]:
b.derivative = function(params, sigma){
  b = params[1]
  r = params[2]

  Zj_ti = exp(-r*compare_ti_uj)*Xj_ti
  #first part of beta derivative
  A = b*Zj_ti
  first_part = sum(sigma*diag(A))
  
  exp.A = exp(A)
  yexp.A = Yj_ti*exp.A
  #denominator of the second part
  denom = colSums(yexp.A)
    
  #numerator of the second part
  num = colSums(yexp.A*Zj_ti)

  #second part of beta derivative
  second_part = sum(sigma*num/denom)

  return (first_part - second_part)
}


In [ ]:
b.derivative(c(0,0.001), sigma)

[1] -48.17366

In [ ]:
library(Matrix)
exp_decay_mod <- function(params_ini=c(0, 0.001), sigma, t, u){
  n <- length(t)
  sigma <- sigma
  #t-u vector #Xi(ti) vector
  compare_ti_ui <- (t-u)
  Xi_ti <- compare_ti_ui
  Xi_ti[Xi_ti <= 0] <- 0
  Xi_ti[Xi_ti > 0 ] <- 1

  #compare ti_uj matrix
  #Xj_ti matrix
  compare_ti_uj <- outer(t,u,FUN=function(x,y)(x-y)) #n*n matrix
  Xj_ti <- compare_ti_uj
  Xj_ti[Xj_ti <= 0] <- 0
  Xj_ti[Xj_ti > 0 ] <- 1

  #Yj_ti matrix
  Yj_ti <- outer(t, t, FUN=function(x,y)(x-y))
  Yj_ti[Yj_ti < 0] <- 0
  Yj_ti[Yj_ti > 0] <- 1
  diag(Yj_ti) <- ifelse(heart1$event==0, 1, 0)

  ###############
  #making Xj_ti and Yj_ti into sparse matrices
  ###############
  Xj_ti = as(Xj_ti,"RsparseMatrix")
  Yj_ti = as(Yj_ti,"RsparseMatrix")
  
  ####################################
  #logLikelihood
  logL <- function(params){
    #n <- length(t) defined before the three functions
    b <- params[1]
    r <- params[2]
  
    Zj_ti <- Xj_ti*exp(-r * (compare_ti_uj))
    Zj_ti = as(Zj_ti,"RsparseMatrix")

    #first part of log Likelihood
    #first_part <- sum( sigma * b * Xi_ti * exp(-r*compare_ti_ui) )
    first_part <- sum( sigma * b * diag(Zj_ti) )
  
    #second part of log Likelihood
    inner_part <- colSums(Yj_ti * exp(b*Zj_ti)) 
    log_part <- log(inner_part)
    #print(inner_part)

    second_part <- sum(sigma*log_part)
    #print(second_part)
  
    return (-first_part + second_part)
  }
  
  ######################################
  #gradient with respect to gamma
  g_gamma = function(params){ 
    #n <- length(t)
    b <- params[1]
    r <- params[2]

    Zj_ti <- Xj_ti*exp(-r * (compare_ti_uj))
    Zj_ti = as(Zj_ti,"RsparseMatrix")

    first_part <- (-1) * sum(sigma * b * diag(Zj_ti) * compare_ti_ui )
  
    num <- colSums(Yj_ti * Xj_ti * exp(b * Zj_ti - r * compare_ti_uj))
    denom <- colSums(Yj_ti * exp(b * Zj_ti))
    sec_part <- b * sum(sigma* (num / denom))
  
    return(first_part + sec_part)
  }

  ########################################
  #gradient with respect to beta
  b.derivative = function(params){
    b = params[1]
    r = params[2]

    Zj_ti = exp(-r*compare_ti_uj)*Xj_ti
    Zj_ti = as(Zj_ti,"RsparseMatrix")
    #first part of beta derivative
    A = b*Zj_ti
    first_part = sum(sigma*diag(Zj_ti))
  
    exp.A = exp(A)
    yexp.A = Yj_ti*exp.A
    #denominator of the second part
    #denom = apply(yexp.A,2,sum)
    denom = colSums(yexp.A)
  
    #numerator of the second part
    #num = apply(yexp.A*Zj_ti,2,sum)
    num = colSums(yexp.A*Zj_ti)

    #second part of beta derivative
    second_part = sum(sigma*num/denom)

    return (first_part - second_part)
  }

  gradients <- function(params){
    #b <- params[1]
    #r <- params[2]
    return ( c(b.derivative(params), g_gamma(params)) )
  }

  #########apply nlminb optimization
  opt <- nlminb(start = params_ini, objective = logL, gradient = gradients, lower=c(-Inf,0),upper=c(+Inf, 1), control = list(step.min=0.01, step.max=1, xf.tol=1e-6))
  #opt <- nlminb(start = params_ini, objective = logL, lower=0, control=list(trace=TRUE))
  
  print(opt$par)
  return (opt)


}

In [ ]:
# Import dataset: use jasa1
data(heart, package="survival")

In [ ]:
heart1 <- heart[-which(heart$event==0 & heart$transplant==0),]
heart1$u = ifelse(heart1$transplant == 0, heart1$stop+1, heart1$start)
head(heart1)

,start,stop,event,age,year,surgery,transplant,id,u
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
1,0,50,1,-17.155373,0.1232033,0,0,1,51
2,0,6,1,3.835729,0.2546201,0,0,2,7
4,1,16,1,6.297057,0.2655715,0,1,3,1
6,36,39,1,-7.737166,0.4900753,0,1,4,36
7,0,18,1,-27.214237,0.6078029,0,0,5,19
8,0,3,1,6.595483,0.7008898,0,0,6,4


In [ ]:
u_s <- heart1$u
t_s <- heart1$stop
n <- length(t)
sigma_s <- heart1$event

In [20]:
exp_decay_mod(c(-1.1471,0.001),sigma=sigma_s, t=t_s, u=u_s)

[1] -1.138099  0.000000


$par
[1] -1.138099  0.000000

$objective
[1] 258.6807

$convergence
[1] 1

$iterations
[1] 2

$evaluations
function gradient 
      10        2 

$message
[1] "false convergence (8)"

In [25]:
library(survival)
summary(coxph(Surv(start, stop, event) ~ transplant, heart1))

Call:
coxph(formula = Surv(start, stop, event) ~ transplant, data = heart1)

  n= 99, number of events= 75 

               coef exp(coef) se(coef)      z Pr(>|z|)    
transplant1 -1.1471    0.3176   0.3012 -3.808  0.00014 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

            exp(coef) exp(-coef) lower .95 upper .95
transplant1    0.3176      3.149     0.176    0.5731

Concordance= 0.585  (se = 0.018 )
Likelihood ratio test= 13.43  on 1 df,   p=2e-04
Wald test            = 14.5  on 1 df,   p=1e-04
Score (logrank) test = 15  on 1 df,   p=1e-04


In [23]:
Surv(jasa1$start, jasa1$stop, jasa1$event)

  [1] (  0.0,  49.0]  (  0.0,   5.0]  (  0.0,  15.0]  (  0.0,  35.0+]
  [5] ( 35.0,  38.0]  (  0.0,  17.0]  (  0.0,   2.0]  (  0.0,  50.0+]
  [9] ( 50.0, 674.0]  (  0.0,  39.0]  (  0.0,  84.0]  (  0.0,  11.0+]
 [13] ( 11.0,  57.0]  (  0.0,  25.0+] ( 25.0, 152.0]  (  0.0,   7.0] 
 [17] (  0.0,  16.0+] ( 16.0,  80.0]  (  0.0,  36.0+] ( 36.0,1386.0] 
 [21] (  0.0,   0.5]  (  0.0,  27.0+] ( 27.0, 307.0]  (  0.0,  35.0] 
 [25] (  0.0,  19.0+] ( 19.0,  42.0]  (  0.0,  36.0]  (  0.0,  17.0+]
 [29] ( 17.0,  27.0]  (  0.0,   7.0+] (  7.0,1031.0]  (  0.0,  11.0+]
 [33] ( 11.0,  50.0]  (  0.0,   2.0+] (  2.0, 732.0]  (  0.0,  82.0+]
 [37] ( 82.0, 218.0]  (  0.0,  24.0+] ( 24.0,1799.0+] (  0.0,1400.0+]
 [41] (  0.0, 262.0]  (  0.0,  70.0+] ( 70.0,  71.0]  (  0.0,  34.0] 
 [45] (  0.0,  15.0+] ( 15.0, 851.0]  (  0.0,  15.0]  (  0.0,  16.0+]
 [49] ( 16.0,  76.0]  (  0.0,  50.0+] ( 50.0,1586.0+] (  0.0,  22.0+]
 [53] ( 22.0,1571.0+] (  0.0,  11.0]  (  0.0,  45.0+] ( 45.0,  99.0] 
 [57] (  0.0,  18.0+